# Affinity Propagation Algorithm Implementation

In [1]:
import numpy as np
import pandas as pd


Creating the dataset

In [47]:
d1 = {}
d1['Participant'] = ['Alice','Bob','Cary','Doug','Edna']
d1['Tax Rate'] = [3,4,3,2,1]
d1['Fee'] = [4,3,5,1,1]
d1['Interest Rate']=[3,5,3,3,3]
d1['Quantity Limit'] = [2,1,3,3,2]
d1['Price Limit'] = [1,1,3,2,3]
df = pd.DataFrame(d1)
df

,Participant,Tax Rate,Fee,Interest Rate,Quantity Limit,Price Limit
0,Alice,3,4,3,2,1
1,Bob,4,3,5,1,1
2,Cary,3,5,3,3,3
3,Doug,2,1,3,3,2
4,Edna,1,1,3,2,3


Mending the data into a matrix

In [2]:
mat=df.iloc[:,1:].values
mat

array([[3, 4, 3, 2, 1],
       [4, 3, 5, 1, 1],
       [3, 5, 3, 3, 3],
       [2, 1, 3, 3, 2],
       [1, 1, 3, 2, 3]], dtype=int64)

Initialising matrices with zero values

In [3]:
#similarity matrix
sim_mat=np.zeros(mat.shape)  
#responsibility matrix
resp_mat=np.zeros(mat.shape)
#availability matrix
avail_mat=np.zeros(mat.shape)
#criterion matrix
crit_mat=np.zeros(mat.shape)

Constructing similarity matrix

In [105]:
sim_mat=np.zeros(mat.shape)
for i in range(mat.shape[0]):
    for j in range(mat.shape[0]):
        if i!=j:
            sim_mat[i,j]=-sum((mat[i]-mat[j])**2)
            
min_sim_mat=min([sim_mat[i][j] for i in range(sim_mat.shape[0]) for j in range(sim_mat.shape[1]) if i!=j])
np.fill_diagonal(sim_mat,min_sim_mat)
sim_mat

array([[-22.,  -7.,  -6., -12., -17.],
       [ -7., -22., -17., -17., -22.],
       [ -6., -17., -22., -18., -21.],
       [-12., -17., -18., -22.,  -3.],
       [-17., -22., -21.,  -3., -22.]])

Defining Responsibility Matrix

In [101]:
def r_mat(avail_mat):  
    for i in range(mat.shape[0]):
            for j in range(mat.shape[0]):
                resp_mat[i,j]=sim_mat[i,j]-max(np.delete(avail_mat[i,:],j)+np.delete(sim_mat[i,:],j))
    return resp_mat
r_mat(avail_mat)  #for the 1st iteration

array([[-16.,  -1.,   1.,  -6., -11.],
       [ 10., -15., -10., -10., -15.],
       [ 11., -11., -16., -12., -15.],
       [ -9., -14., -15., -19.,   9.],
       [-14., -19., -18.,  14., -19.]])

Defining availability matrix

In [102]:
def a_mat(resp_mat):
    for i in range(avail_mat.shape[1]):
        for j in range(avail_mat.shape[1]):
            if i == j:
                avail_mat[i,j]=sum(resp_mat[:,i][resp_mat[:,i]>0])
            else:
                nn=sum([resp_mat[k,j] for k in range(resp_mat.shape[1]) if k!=j and k!=i and resp_mat[k,j]>0])
                avail_mat[i,j]=min(0,resp_mat[j,j]+nn)
    return avail_mat
a_mat(resp_mat)  #for the first iteration

array([[ 21., -15., -16.,  -5., -10.],
       [ -5.,   0., -15.,  -5., -10.],
       [ -6., -15.,   1.,  -5., -10.],
       [  0., -15., -15.,  14., -19.],
       [  0., -15., -15., -19.,   9.]])

Parameters defined

In [61]:
d=0.5 #bit of noise to assure convergence
t=15 #number of iterations

Iterations to get criterion matrix

In [ ]:
for i in range(t):
    if i==0:
        r_dash=r_mat(avail_mat)
        a_dash=a_mat(resp_mat)
    else:
        r_dash=d*r_dash+(1-d)*r_mat(a_dash)
        a_dash=d*a_dash+(1-d)*a_mat(r_dash)

In [62]:
c_dash=r_dash+a_dash
c_dash

array([[ 12., -22., -22., -12., -17.],
       [ 10., -15., -26., -10., -15.],
       [ 12., -26., -16., -12., -15.],
       [ -2., -22., -24.,   0.,   0.],
       [ -2., -22., -22.,   0.,   0.]])

Cluster Formation

In [104]:
clstr=[]
for i in range(c_dash.shape[0]):
    cls=int(np.arange(c_dash.shape[0])[c_dash[i,:]==max(c_dash[i,:])][0]) 
    clstr.append(cls)
print("clusters=", clstr)

clusters= [0, 0, 0, 3, 3]
